In [1]:
import pandas as pd
import sqlite3

In [2]:
DB_NAME = '../data/checking-logs.sqlite'

conn = sqlite3.connect(DB_NAME)

conn.execute("""DROP TABLE IF EXISTS datamart;""")

query ="""               
CREATE TABLE datamart AS
SELECT 
    checker.uid, 
    checker.labname, 
    MIN(checker.timestamp) AS first_commit_ts, 
    MIN(pageviews.datetime) AS first_view_ts 
FROM 
    checker
LEFT JOIN 
    pageviews ON checker.uid = pageviews.uid
WHERE 
    checker.uid LIKE 'user_%'
    AND checker.status = 'ready'
    AND checker.numTrials = 1
    AND checker.labname IN ('laba04','laba04s','laba05','laba06','laba06s','project1')
GROUP BY checker.uid, checker.labname;
"""

conn.execute(query)
conn.commit()

query_fetch = "SELECT * FROM datamart;"
datamart = pd.read_sql_query(query_fetch, conn)
# df = pd.read_sql_query(query_fetch,conn,parse_dates=['first_commit_ts','first_view_ts'])
datamart['first_commit_ts'] = pd.to_datetime(datamart['first_commit_ts'])
datamart['first_view_ts'] = pd.to_datetime(datamart['first_view_ts'])


In [3]:
datamart.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 140 entries, 0 to 139
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   uid              140 non-null    object        
 1   labname          140 non-null    object        
 2   first_commit_ts  140 non-null    datetime64[ns]
 3   first_view_ts    59 non-null     datetime64[ns]
dtypes: datetime64[ns](2), object(2)
memory usage: 4.5+ KB


In [4]:
datamart.dtypes

uid                        object
labname                    object
first_commit_ts    datetime64[ns]
first_view_ts      datetime64[ns]
dtype: object

In [5]:
test = datamart[datamart['first_view_ts'].notnull()].copy()
#test_users

In [6]:
control = datamart[datamart['first_view_ts'].isnull()].copy()
control

,uid,labname,first_commit_ts,first_view_ts
12,user_11,laba05,2020-05-03 21:06:55.970293,NaT
13,user_11,project1,2020-05-03 23:45:33.673409,NaT
14,user_12,laba04,2020-04-18 17:07:51.767358,NaT
15,user_12,laba04s,2020-04-26 15:42:38.070593,NaT
16,user_12,laba05,2020-05-03 08:39:25.174316,NaT
...,...,...,...,...
135,user_8,laba04s,2020-04-19 10:22:35.761944,NaT
136,user_8,laba05,2020-05-02 13:28:07.705193,NaT
137,user_8,laba06,2020-05-16 17:56:15.755553,NaT
138,user_8,laba06s,2020-05-16 20:01:07.900727,NaT


In [7]:
avg_first_view_ts = test['first_view_ts'].mean()
avg_first_view_ts

Timestamp('2020-04-27 00:40:05.761783552')

In [8]:
control['first_view_ts'] = control['first_view_ts'].fillna(avg_first_view_ts)
control

,uid,labname,first_commit_ts,first_view_ts
12,user_11,laba05,2020-05-03 21:06:55.970293,2020-04-27 00:40:05.761783552
13,user_11,project1,2020-05-03 23:45:33.673409,2020-04-27 00:40:05.761783552
14,user_12,laba04,2020-04-18 17:07:51.767358,2020-04-27 00:40:05.761783552
15,user_12,laba04s,2020-04-26 15:42:38.070593,2020-04-27 00:40:05.761783552
16,user_12,laba05,2020-05-03 08:39:25.174316,2020-04-27 00:40:05.761783552
...,...,...,...,...
135,user_8,laba04s,2020-04-19 10:22:35.761944,2020-04-27 00:40:05.761783552
136,user_8,laba05,2020-05-02 13:28:07.705193,2020-04-27 00:40:05.761783552
137,user_8,laba06,2020-05-16 17:56:15.755553,2020-04-27 00:40:05.761783552
138,user_8,laba06s,2020-05-16 20:01:07.900727,2020-04-27 00:40:05.761783552


In [9]:
conn.execute("DROP TABLE IF EXISTS test;")
conn.execute("DROP TABLE IF EXISTS control;")


In [10]:
test.to_sql('test',conn, if_exists='replace',index=False)
control.to_sql('control',conn,if_exists='replace',index=False)
test.info()


<class 'pandas.core.frame.DataFrame'>
Index: 59 entries, 0 to 114
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   uid              59 non-null     object        
 1   labname          59 non-null     object        
 2   first_commit_ts  59 non-null     datetime64[ns]
 3   first_view_ts    59 non-null     datetime64[ns]
dtypes: datetime64[ns](2), object(2)
memory usage: 2.3+ KB


In [11]:
control.info()

<class 'pandas.core.frame.DataFrame'>
Index: 81 entries, 12 to 139
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   uid              81 non-null     object        
 1   labname          81 non-null     object        
 2   first_commit_ts  81 non-null     datetime64[ns]
 3   first_view_ts    81 non-null     datetime64[ns]
dtypes: datetime64[ns](2), object(2)
memory usage: 3.2+ KB


In [12]:
conn.commit()
conn.close()